In [1]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
#================================================================#
#============= Proceso de extraccion y tratamiento ==============#
#================================================================#

#====================== Import de librerias =====================#

import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen
import datetime
import plotly.express as px
import plotly.graph_objects as go


#============ Definicion de valores de configuracion ============#

# Url de dataset que queremos descargar
urls=[
  "https://jmcauley.ucsd.edu/data/amazon_v2/categoryFilesSmall/Musical_Instruments.csv",
  "https://drive.google.com/file/d/1mk1UXlKn90nF2c3Ok54kbyKlPuV9B15T/view?usp=share_link"
]

# Nombre de los archivos
files=[
  "Musical_Instruments.csv",
  "meta_Musical_Instruments.json.gz"
]

# Path de datos
path_data= "/content/drive/MyDrive/postgraduate/Trabajo_final/data"

# Minimo de reviews por usuario
min_reviews=6
min_usuarios=9

# Nombre de columnas
col_names = {"col_id_product": "asin",
             "col_id_reviewer": "reviewerID",
             "col_unix_time": "timestamp",
             "col_year": "year",
             "col_rating": "overall"}

#============ Definicion de funciones de extraccion y tratamiento ============#

def helper(arg):

   '''
   Funcion auxiliar para dar informacion del proceso de descarga
   '''

   if(os.system(arg) == 0):
        print(arg," completed")
        return None

def get_dataset_basic_info(df, nombre):

  '''
  Funcion que retorna infomracion de # de clientes, productos y reviews en dataset elegido
  '''

  aux=len(nombre)
  numero_clientes=len(df[col_names["col_id_reviewer"]].unique())
  numero_productos=len(df[col_names["col_id_product"]].unique())

  return(print("\n","#"*aux,"\n",nombre.center(aux),"\n","#"*aux,"\n"*2, "Numero total de clientes: ", numero_clientes,"\n",
      "Numero total de productos: ", numero_productos, "\n",
      "Numero total de reviews: ", df.shape[0],"\n"))


def load_data_raw(download=False):

  '''
  Funcion que descarga los datasets de las url elegidas. 
  '''

  cols = [col_names["col_id_product"], col_names["col_id_reviewer"], col_names["col_rating"], col_names["col_unix_time"]]

  ### Descarga de información
  if download==True:
    for i,url in enumerate(urls):
      helper("wget -O "+path_data+"/"+files[i]+" " +url+ " --no-check-certificate")

  for i,archivo in enumerate(files):
    if "csv" in archivo:
      df = pd.read_csv(path_data+"/"+archivo, delimiter=",", names=cols)
    elif "json" in archivo:    
      meta = []
      with gzip.open(path_data+"/"+archivo) as f:
          for l in f:
              meta.append(json.loads(l.strip()))

      meta = pd.DataFrame.from_dict(meta)

  return df, meta


def treat_dataset_src(df, min_reviews,min_usuarios,info=False):

    '''
    Funcion que trata el dataset original
    Realiza conversiones de tipo
    Filtra duplicados (usuarios que han puesto mas de una review sobre un producto el mismo día)
    Filtra productos comprados por al menos 3 usuarios
    Filtra solo usuarios con mas de X reviews
    '''

    # Conversion de tipo
    df[col_names["col_rating"]] = pd.to_numeric(df[col_names["col_rating"]].replace(',','', regex=True))
    df[col_names["col_unix_time"]]=df[col_names["col_unix_time"]].apply(lambda x: datetime.datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
    df[col_names["col_year"]]= pd.to_datetime(df[col_names["col_unix_time"]]).dt.year

    # Duplicados
    df_duplicates = df[[col_names["col_id_reviewer"],col_names["col_id_product"], col_names["col_unix_time"]]].sort_values(by=[col_names["col_unix_time"]], ascending=False)
    df = df.drop(df_duplicates[df_duplicates[[col_names["col_id_reviewer"],col_names["col_id_product"]]].duplicated()][col_names["col_id_reviewer"]].index.values.tolist())
   
    if (info==True):
      get_dataset_basic_info(df,"Informacion tras eliminar duplicados")

    productos_a_eliminar=[1]
    clientes_a_eliminar=[1]
    iteracion = 1
    while (len(productos_a_eliminar)!=0)and(len(clientes_a_eliminar)!=0):

      # Minimo de usuarios que han comprado el producto productos 
      aux=df.groupby([col_names["col_id_product"]])[col_names["col_id_reviewer"]].count().reset_index()
      aux2=aux[aux[col_names["col_id_reviewer"]]<min_usuarios].reset_index() # usuarios a eliminar
      aux=aux[aux[col_names["col_id_reviewer"]]>=min_usuarios].reset_index() # usuarios a conservar
      productos=aux[col_names["col_id_product"]]
      df=df[df[col_names["col_id_product"]].isin(productos)]

      productos_a_eliminar=aux2[col_names["col_id_product"]]


      if (info==True):
        get_dataset_basic_info(df,f"Iteracion: {iteracion}. Informacion tras eliminar productos comprados por menos de {min_usuarios} personas")

      # Seleccionamos los ids de producto que tienen mas de X reviews 
      aux=df.groupby([col_names["col_id_reviewer"]])[col_names["col_rating"]].count().reset_index()
      aux2=aux[aux[col_names["col_rating"]]<min_reviews].reset_index()
      aux=aux[aux[col_names["col_rating"]]>=min_reviews].reset_index()
      clientes=aux[col_names["col_id_reviewer"]]
      df=df[df[col_names["col_id_reviewer"]].isin(clientes)]

      clientes_a_eliminar=aux2[col_names["col_id_reviewer"]]

      if (info==True):
        get_dataset_basic_info(df,f"Iteracion: {iteracion}. Informacion tras eliminar usuarios con menos de {min_reviews} reviews")

      iteracion+=1

    return df


def treat_metadata(meta, products):

  '''
  Funcion que trata el dataset de metadatos original
  Realiza un filtro para tener los productos presentes en el dataset de reviews tratado
  '''

  meta=meta[meta[col_names["col_id_product"]].isin(products)]

  meta=meta.drop_duplicates(subset="asin",keep='first')

  return meta






#============ Visualización ============#

def barplot_reviews(df):

  '''
  Visualizaciones interesantes:
  Barplot con el numero de usuarios en funcion del numero de reviews en dataset
  '''

  # Definicion de dataset
  data = pd.DataFrame(df.groupby([col_names["col_id_reviewer"]]).count()).reset_index().groupby([col_names["col_rating"]]).count().reset_index()

  # Definicion del objeto fig
  fig = go.Figure()

  # Gráfica
  fig.add_trace(go.Bar(x=data[col_names["col_rating"]],
                  y=data[col_names["col_id_reviewer"]],
                  name='Rest of world',
                  marker_color='rgb(55, 83, 109)'
                  ))


  fig.update_layout(
      title='¿Cuantos usuarios tienen x reviews?',
      xaxis_tickfont_size=14,
      yaxis=dict(
          title='# de usuarios',
          titlefont_size=16,
          tickfont_size=14,
      ),
      xaxis=dict(
          title='# de reviews',
          titlefont_size=16,
          tickfont_size=14,
      )
  )

  return(fig.show())

def timeline_reviews(df):

  # Definicion de dataset
  data = pd.DataFrame(df.groupby([col_names["col_year"]]).count()).reset_index()

  # Definicion de objeto figura
  fig = go.Figure()

  # Definicion de grafica
  fig.add_trace(go.Bar(x=data[col_names["col_year"]],
                  y=data[col_names["col_rating"]],
                  name='Rest of world',
                  marker_color='rgb(55, 83, 109)'
                  ))


  fig.update_layout(
      title='¿Cuantas reviews tenemos por año?',
      xaxis_tickfont_size=14,
      yaxis=dict(
          title='# de usuarios',
          titlefont_size=16,
          tickfont_size=14,
      ),
      xaxis=dict(
          title='Año',
          titlefont_size=16,
          tickfont_size=14,
      )
  )

  return(fig.show())

In [19]:
df, meta=load_data_raw(download=False)

In [22]:
get_dataset_basic_info(df,"Descarga de la información en bruto RAW")
df1=treat_dataset_src(df, min_reviews,min_usuarios, True)
get_dataset_basic_info(df1,"Definición del dataset inicial tras proceamiento SRC")


 #################################### 
 Informacion tras eliminar duplicados 
 #################################### 

 Numero total de clientes:  903330 
 Numero total de productos:  112222 
 Numero total de reviews:  1470564 


 ################################################################################### 
 Iteracion: 1. Informacion tras eliminar productos comprados por menos de 9 personas 
 ################################################################################### 

 Numero total de clientes:  801368 
 Numero total de productos:  23162 
 Numero total de reviews:  1252836 


 ####################################################################### 
 Iteracion: 1. Informacion tras eliminar usuarios con menos de 6 reviews 
 ####################################################################### 

 Numero total de clientes:  20454 
 Numero total de productos:  19631 
 Numero total de reviews:  203491 


 #####################################################################

In [55]:
meta_treated=treat_metadata(meta, df1[col_names["col_id_product"]].unique())

In [57]:
from datetime import date

today = date.today()

df1.to_pickle(path_data+"/"+"SRC_Musical_instruments"+"_"+today.strftime("%d%m%Y")+".pkl")
meta_treated.to_pickle(path_data+"/"+"SRC_meta_Musical_instruments"+"_"+today.strftime("%d%m%Y")+".pkl")